In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('max_columns', None)

In [161]:
data_raw = pd.read_csv('train.csv')
data_for_test = pd.read_csv('test.csv')

In [162]:
data_raw.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [163]:
numerical_attr = ['Age','SibSp','Parch','Fare',]
categorical_attr = ['Pclass','Sex','Embarked',]
drop_attr = ['PassengerId','Ticket','Cabin']
text_attr = ['Name']

labels = data_raw['Survived']

In [164]:
X = data_raw.drop('Survived', axis=1)

In [165]:
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [166]:
# def add_att(f,g):
#     if (type(f)!= np.number) or (type(g) != np.number):
#         raise TypeError('f or g is not float type')
#     e = np.add(f,g)
#     if (type(e)!= np.number):
#         raise TypeError('f or g is not float type')
#     e = np.add(e, np.ones(891))
#     return e
    
# # vfunc = np.vectorize(add_att)
# f = X.iloc[:,6]
# g = X.iloc[:,5]

# f.dtype
# # family_mems = add_att(f,g)
# # len(f)
# # family_mems

In [167]:
# f = X.iloc[:,6]
# f

In [168]:
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [336]:
from sklearn.preprocessing import FunctionTransformer

def add_extra_features(X):
    
    print(X.columns)
    
    family_mems = X.iloc[:, 1] + X.iloc[:, 2] + 1
    
    print("Done family_mems")
    
    def decide_marital_status(g):
        if g == 1:
            return 'Single'
        elif g == 2:
            return 'Couple'
        elif g>2:
            return 'Family'
    # vfunc = np.vectorize(decide_marital_status)
    # marital_status = vfunc(decide_marital_status)
    
    marital_status = np.array([decide_marital_status(i) for i in family_mems])
    
    print("Done marital_status")
    
    def decide_age_group(a):
        a = int(a)
        if a<=5:
            return 'Infant'
        elif 5<a<=13:
            return 'Child'
        elif 13<a<26:
            return 'Youth'
        else:
            return 'Adult'
    if X.iloc[:,0].isna().sum():
        raise ValueError("There are Nan values in 'age'")
    age_group = np.array([decide_age_group(i) for i in X.iloc[:,0]])
    
    print("Done age_group")
    
    marital_status = marital_status.reshape(891,1)
    age_group = age_group.reshape(891,1)
    family_mems = np.array(family_mems).reshape(891,1)
    
    print("Shape of X ", X.shape)
    print("Shape of family_mems ", family_mems.shape)
    print("Shape of marital-status ", marital_status.shape)
    print("Shape of age_group ", age_group.shape)
    
    xyz = np.concatenate((X, family_mems, marital_status, age_group),axis=1)
    return xyz

attr_adder = FunctionTransformer(add_extra_features, validate=False,)
attr_adder.fit(X)


FunctionTransformer(func=<function add_extra_features at 0x7f30c15fb130>)

In [307]:
X['Age'].value_counts()

24.00    30
22.00    27
18.00    26
19.00    25
28.00    25
         ..
36.50     1
55.50     1
0.92      1
23.50     1
74.00     1
Name: Age, Length: 88, dtype: int64

In [337]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

pl_text = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
])
all_cols = numerical_attr + categorical_attr + text_attr
imputer = SimpleImputer(strategy='most_frequent')
print(X.shape)
print(data_raw.shape)
X_im = imputer.fit_transform(X)
X_im.shape

X_im = pd.DataFrame(X_im, columns = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], index =X.index)


ct = ColumnTransformer([
    ('attr_adder', attr_adder, all_cols),
    ('text_tr', pl_text, text_attr),
    ('onehot', OneHotEncoder(sparse=False), categorical_attr),
    ('scaling', StandardScaler(), numerical_attr),
])

(891, 11)
(891, 12)


In [335]:
age = X.Age.copy()

age = np.array(age).reshape(891,1)

print(age.shape)
type(age)

(891, 1)


numpy.ndarray

In [338]:
datav1 = ct.fit_transform(X_im)


Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'Sex', 'Embarked', 'Name'], dtype='object')
Done family_mems
Done marital_status
Done age_group
Shape of X  (891, 8)
Shape of family_mems  (891, 1)
Shape of marital-status  (891, 1)
Shape of age_group  (891, 1)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 891 and the array at index 1 has size 1